In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch as torch
import torch.nn as nn

In [3]:
input_x1 = pd.read_csv('df1_smallestdata.csv')
input_x2 = pd.read_csv('df2_smallestdata.csv')
target_y = pd.read_csv('targets_smallestdata.csv')
 
# Drop the first column
input_x1 = input_x1.drop(input_x1.columns[0], axis=1)
input_x2 = input_x2.drop(input_x2.columns[0], axis=1)

# Convert to numpy
input_x1 = input_x1.to_numpy()
input_x2 = input_x2.to_numpy()
target_y = target_y.to_numpy()

print(input_x1.shape)
print(input_x2.shape)
print(target_y.shape)



(250, 24)
(250, 24)
(250, 1)


In [4]:
input_x1

array([[2., 1., 0., ..., 0., 0., 0.],
       [0., 5., 1., ..., 1., 0., 0.],
       [0., 2., 1., ..., 0., 0., 0.],
       ...,
       [1., 2., 0., ..., 0., 2., 0.],
       [2., 1., 0., ..., 0., 0., 0.],
       [0., 4., 0., ..., 0., 0., 0.]])

In [2]:
# make a neural network with 1 hidden layer using pytorch
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(20, 16)
        self.fc2 = nn.Linear(16, 9)
        self.fc3 = nn.Linear(9, 3)
        self.fc4 = nn.Linear(3, 1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.criterion = nn.MSELoss()
        self.optimizer = torch.optim.SGD(self.parameters(), lr=0.01)



    def forward(self, x1, x2):
        # Element-wise subtraction
        x = x1 - x2
        # Forward pass with shared weights
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        x = self.fc4(x)
        return x
    
    def loss(self, output, target):
        return self.criterion(output, target)
    
    def backward(self, loss):
        loss.backward()

    def train(self, x1, x2, target):
        # Forward pass
        x1 = torch.tensor(x1, dtype=torch.float32)
        x2 = torch.tensor(x2, dtype=torch.float32)
        output = self.forward(x1, x2)
        # Compute the loss   
        target = torch.tensor(target, dtype=torch.float32)
        loss = self.loss(output, target)
        # Zero the gradients
        self.optimizer.zero_grad()
        # Backward pass
        self.backward(loss)
        # Update the weights
        self.optimizer.step()
        return loss

    

In [3]:
model = torch.load("trained_sample.pth")

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [8]:
def setup(input_x1, input_x2, target_y):
    # Create a neural network
    net = Net()
    # Train the network
    for i in range(100):
        for j in range(len(input_x1)):
            net.train(input_x1[j], input_x2[j], target_y[j])
    return net

In [9]:
network = setup(input_x1, input_x2, target_y) # 2 epochs took 22mins to run

In [8]:
for i in range(10):
    print(network.forward(torch.tensor(input_x1[i], dtype=torch.float32), torch.tensor(input_x2[i], dtype=torch.float32)))
    print(target_y[i])
    print('\n')


tensor([-0.0015], grad_fn=<ViewBackward0>)
[0 1]


tensor([0.9985], grad_fn=<ViewBackward0>)
[1 0]


tensor([1.0023], grad_fn=<ViewBackward0>)
[1 0]


tensor([0.9987], grad_fn=<ViewBackward0>)
[1 0]


tensor([0.0008], grad_fn=<ViewBackward0>)
[0 1]


tensor([-0.0014], grad_fn=<ViewBackward0>)
[0 1]


tensor([0.9979], grad_fn=<ViewBackward0>)
[1 0]


tensor([0.0029], grad_fn=<ViewBackward0>)
[0 1]


tensor([-0.0013], grad_fn=<ViewBackward0>)
[0 1]


tensor([0.0021], grad_fn=<ViewBackward0>)
[0 1]


